<a href="https://colab.research.google.com/github/coll-j/IndonesianDepParse/blob/master/gegem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [1]:
%%bash
wget https://github.com/coll-j/IndonesianDepParse/blob/master/test.txt
pip install nlp-id

  Created wheel for nlp-id: filename=nlp_id-0.1.9.8-cp36-none-any.whl size=7723018 sha256=7f66d4f55cc59e6dce7bf18c144c142ebdc517f35f58ad105fb9efc921c34edf
  Stored in directory: /root/.cache/pip/wheels/87/04/4e/25d7de62878a993a6642138ef4c9b5ea2cdc78f12cf8624c4d
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=6e15a0f0707452164c94e18cd40754b8dcca00ab44b6027aadcf6b4f2ffa02c1
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=16e2dd59555f921341b21db1778ba77a550d2f1ea670552a39cb4ad7c841b5d4
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built nlp-id nltk wget
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: nltk 3

--2020-08-23 13:08:01--  https://github.com/coll-j/IndonesianDepParse/blob/master/test.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘test.txt’

     0K .......... .......... .......... .......... ..........  920K
    50K .......... .......... .........                        1.37M=0.08s

2020-08-23 13:08:02 (1.03 MB/s) - ‘test.txt’ saved [81761]



In [ ]:
import nltk
nltk.download('punkt')

from nlp_id.tokenizer import Tokenizer
from nlp_id.postag import PosTag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
FIELD_TO_IDX = {'idx': 0, 'word': 1, 'postag': 2, 'head': 3, 'deprel': 4}
def get_long_tensor(tokens_list, batch_size):
    """ Convert (list of )+ tokens to a padded LongTensor. """
    sizes = []
    x = tokens_list
    size = (max(len(y) for y in x))
    tokens = torch.LongTensor(batch_size, size).fill_(0)
    for i, s in enumerate(tokens_list):
      ten = torch.LongTensor(list(map(ord, s[:len(s)])))
      tokens[i, :len(s)] = ten
    return tokens

In [3]:
class Loader:
  
  def __init__(self, file_path, batch_size):
    self._file_path = file_path
    self.batch_size = batch_size

    self.data = self.load_file()
    self.vocabs = self.get_vocabs(self.data)
    self.data = self.chunk_batches(self.data)

  def load_file(self):
    sents, sent = [], []
    f = open(self._file_path)
    for line in f:
      line = line.strip()
      if len(line) == 0:
        if len(sent) > 0:
          sents.append(sent)
          sent = []

      else:
        cols = line.split('\t')
        sent += [cols]
    if len(sent) > 0:
      sents.append(sent)

    return sents

  def get_vocabs(self, data):
    words = [cols[FIELD_TO_IDX['word']] for sent in data for cols in sent]
    postags = [cols[FIELD_TO_IDX['postag']] for sent in data for cols in sent] 
    deprels = [cols[FIELD_TO_IDX['deprel']] for sent in data for cols in sent] 

    words = self.build_vocab(words)
    postags = self.build_vocab(postags)
    deprels = self.build_vocab(deprels)

    return {'words': words, 'postags': postags, 'deprels': deprels}

  def build_vocab(self, cols):
    w2i = {}
    for w in cols:
      if w not in w2i:
        w2i[w] = len(w2i)
    
    return w2i

  def chunk_batches(self, data):
    # TO DO: Chunk data into batches
    res, curr = [], []
    currlen = 0
    for sent in data:
      if len(sent[0]) + currlen > self.batch_size:
        if len(curr) > 0:
          res.append(curr)
          curr = []
          currlen = 0
      
      curr.append(sent)
      currlen += len(sent[0])
    if len(curr) > 0:
      res.append(curr)

    return res
    
  def get_batches(self, key):
    print('data[i]', self.data[key])
    batch = self.data[key]
    # convert to tensors
    words = [cols[FIELD_TO_IDX['word']] for sent in batch for cols in sent]
    words = get_long_tensor(words, len(words))
    words_mask = torch.eq(words, 0)

    postags = [cols[FIELD_TO_IDX['postag']] for sent in batch for cols in sent]
    postags = get_long_tensor(postags, len(postags))
    heads = [cols[FIELD_TO_IDX['head']] for sent in batch for cols in sent]
    heads = get_long_tensor(heads, len(heads))
    deprels = [cols[FIELD_TO_IDX['deprel']] for sent in batch for cols in sent]
    deprels = get_long_tensor(deprels, len(deprels))
    sentlens = [len(y) for y in batch]
    print('sentlens', sentlens)

    return words, words_mask, postags, heads, deprels, sentlens

  def __iter__(self):
    for i in range(len(self.data)):
      yield self.get_batches(i)

In [ ]:
data = Loader('test.txt', 11)
for d in data:
  print(d)

data[i] [[['1', 'Banyak', 'NUM', '2', 'nummod'], ['2', 'buku', 'NN', '5', 'obj'], ['3', 'telah', 'ADV', '5', 'advmod'], ['4', 'saya', 'PR', '5', 'nsubj'], ['5', 'baca', 'VB', '0', 'root']], [['1', 'Buku', 'NN', '4', 'nsubj'], ['2', 'ini', 'PR', '1', 'nmod'], ['3', 'telah', 'ADV', '4', 'advmod'], ['4', 'dibaca', 'VB', '0', 'root']]]
sentlens [5, 4]
(tensor([[ 66,  97, 110, 121,  97, 107],
        [ 98, 117, 107, 117,   0,   0],
        [116, 101, 108,  97, 104,   0],
        [115,  97, 121,  97,   0,   0],
        [ 98,  97,  99,  97,   0,   0],
        [ 66, 117, 107, 117,   0,   0],
        [105, 110, 105,   0,   0,   0],
        [116, 101, 108,  97, 104,   0],
        [100, 105,  98,  97,  99,  97]]), tensor([[False, False, False, False, False, False],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, Fals

In [ ]:
data.vocabs

{'deprels': {'advmod': 2,
  'appos': 6,
  'nmod': 5,
  'nsubj': 3,
  'nummod': 0,
  'obj': 1,
  'punct': 7,
  'root': 4},
 'postags': {'ADV': 2, 'IN': 5, 'NN': 1, 'NUM': 0, 'PR': 3, 'SYM': 6, 'VB': 4},
 'words': {'.': 14,
  'Banyak': 0,
  'Buku': 5,
  'Kita': 8,
  'baca': 4,
  'berada': 10,
  'buku': 1,
  'dalam': 12,
  'di': 11,
  'dibaca': 7,
  'ini': 6,
  'ruangan': 13,
  'saya': 3,
  'sedang': 9,
  'telah': 2}}

In [ ]:
data.data

[[[['1', 'Banyak', 'NUM', '2', 'nummod'],
   ['2', 'buku', 'NN', '5', 'obj'],
   ['3', 'telah', 'ADV', '5', 'advmod'],
   ['4', 'saya', 'PR', '5', 'nsubj'],
   ['5', 'baca', 'VB', '0', 'root']],
  [['1', 'Buku', 'NN', '4', 'nsubj'],
   ['2', 'ini', 'PR', '1', 'nmod'],
   ['3', 'telah', 'ADV', '4', 'advmod'],
   ['4', 'dibaca', 'VB', '0', 'root']]],
 [[['1', 'Kita', 'PR', '3', 'nsubj'],
   ['2', 'sedang', 'ADV', '3', 'advmod'],
   ['3', 'berada', 'VB', '0', 'root'],
   ['4', 'di', 'IN', '6', 'appos'],
   ['5', 'dalam', 'NN', '6', 'advmod'],
   ['6', 'ruangan', 'NN', '3', 'obj'],
   ['7', '.', 'SYM', '3', 'punct']]]]

# Build Model

In [ ]:
class Model(nn.Module):
  def __init__(self, args, vocab):
    super().__init__()

    self.args = args

    # input layer
    input_size = 0
    self.wordemb = nn.Embedding(len(vocab['words']), self.args['word_emb_dim'])
    input_size += self.args['word_emb_dim']
    self.posemb = nn.Embedding(len(vocab['postags']), self.args['pos_emb_dim'])
    input_size += self.args['pos_emb_dim']
    self.depemb = nn.Embedding(len(vocab['deprels']), self.args['dep_emb_dim'])

    # recurrent layer
    self.parserGRU = nn.GRU(input_size, self.args['hidden_dim'], self.args['num_layers'],\
                            batch_first=True, dropout=self.args['dropout'], bidirectional=True)
    self.GRU_h0 = nn.Parameter(torch.zeros(self.args['num_layers'] * 2),\
                               self.args['batch_size'], self.args['hidden_dim'])
    
    # classifier
    # TO DO: add MLP and Deep Biaffine for each head and deprel

    # criterion
    self.criterion = nn.CrossEntropyLoss()
    
    self.dropout = nn.Dropout(self.args['dropout'])
  def forward(self):
    #TODO: make forward func
    return